# Seminar 4: Data Formats and APIs

November 11, 2025

______________________________

## What are we going to learn

- What are the most popular data formats and how to work with them
- Databases, SQL and data manipulation
- HTTP requests
- How to scrape websites 

## 1. Data formats

### Data Serialization and Formats
Serialization is the process of converting data structures or objects into a format that can be easily stored and shared. Common data formats in Python include:
- **JSON**: Lightweight and commonly used for APIs.
- **CSV**: Ideal for tabular data, especially in spreadsheets.
- **Excel**: Classic data storage everyone is aware of.

#### JSON

In [1]:
import json
import pandas as pd
import os
    
data = {
    "name": "Alice",
    "age": 30,
    "is_member": True,
    "hobbies": ["reading", "biking", "coding"]
}

In [9]:
filepath = os.path.join('Data', 'sample.json')

with open(filepath, 'w') as f:
    json.dump(data, f)

In [13]:
pd.read_json(filepath)

,name,age,is_member,hobbies
0,Alice,30,True,reading
1,Alice,30,True,biking
2,Alice,30,True,coding


#### CSV

In [15]:
import csv

data = [
    ["Name", "Age", "Occupation"],
    ["Alice", 30, "Engineer"],
    ["Bob", 25, "Designer"],
    ["Charlie", 35, "Teacher"]
]


In [17]:
filepath = os.path.join('Data', 'sample.csv')

with open(filepath, 'w') as f:
    writer = csv.writer(f)
    writer.writerows(data)

In [18]:
pd.read_csv(filepath)

,Name,Age,Occupation
0,Alice,30,Engineer
1,Bob,25,Designer
2,Charlie,35,Teacher


#### Excel

In [19]:
data = {
    "name": "Thomas",
    "age": 30,
    "is_member": True,
    "hobbies": "reading, biking, coding"
}

In [24]:
filepath = os.path.join('Data', 'sample.xlsx')
df = pd.DataFrame([data])
df.to_excel(filepath, index=False)

In [25]:
pd.read_excel(filepath)

,name,age,is_member,hobbies
0,Thomas,30,True,"reading, biking, coding"


## 2. Requests and web scraping

In [26]:
import requests # for making HTTP requests
import pandas as pd 
import time
import re # Regex = Regular Expressions

Time package small things:

In [27]:
%%time
print("Hello, World!")

Hello, World!
CPU times: user 491 μs, sys: 490 μs, total: 981 μs
Wall time: 619 μs


In [28]:
t0 = time.time()
time.sleep(1)
t1 = time.time()
print("Time elapsed: ", t1-t0, " seconds")

Time elapsed:  1.0053067207336426  seconds


In [29]:
%%time 
time.sleep(2)

CPU times: user 853 μs, sys: 1.35 ms, total: 2.2 ms
Wall time: 2 s


In [30]:
import random

In [31]:
%%time
r_time = random.uniform(0.5, 1.2)
print("Sleeping for ", r_time, " seconds")
time.sleep(r_time)

Sleeping for  0.5280115277270837  seconds
CPU times: user 2.7 ms, sys: 2.46 ms, total: 5.16 ms
Wall time: 530 ms


#### Task 1: Requesting API

Let us work with data of sreality.cz which we can access via their api. An intuition is that the api is limited for a number of requests (but not verified).

##### 1a. Create a function requesting data from sreality

```python
base_url = 'https://www.sreality.cz/api/cs/v2/estates?category_main_cb=1&category_type_cb=1&locality_region_id=10&per_page60&page={}'.format(i)

r = requests.get(base_url)
d = r.json()
```

0) function should parametrize: 
    * `category_main_cb` - `{'flat':1, 'house':2, 'land':3 }`
    * `category_type_cb` - `{'sell':1,'rent':2}`
    * `locality_region_id` - use 10 as default value
    * `page` parameter
1) use string inputs for `category_main_cb` and `category_type_cb`
2) include `try/except` clause to handle errors
3) function should return JSON data in python types
4) do not forget to sleep each request at least 0.5s

In [45]:
def request_sreality(page, category_main_str, category_type_str, locality_region_id=10):
    """
    Request data from sreality.cz API
    :param page: page number
    :param category_main_str: category of the property
    :param category_type_str: type of the offer
    :param locality_region_id: region id
    :return json: json response
    """
    template_url = 'https://www.sreality.cz/api/cs/v2/estates?category_main_cb={category_main}&category_type_cb={category_type}&locality_region_id={locality_region_id}&per_page60&page={page}'
    category_main_cb = {'flat':1, 'house':2, 'land':3 }
    category_type_cb = {'sell':1,'rent':2}
    url_path = template_url.format(category_main=category_main_cb[category_main_str],
                                   category_type=category_type_cb[category_type_str],
                                   locality_region_id=locality_region_id,
                                   page=page)
    time.sleep(random.uniform(0.5, 1.2))
    try:
        r = requests.get(url_path)
    except Exception as e:
        print(e)
        return None
    return r.json()


In [46]:
d = request_sreality(page=0, category_main_str='flat', category_type_str='sell', locality_region_id=10)

Inspect the element `d`:

In [49]:
d.keys()

dict_keys(['meta_description', 'result_size', '_embedded', 'filterLabels', 'title', 'filter', '_links', 'locality', 'locality_dativ', 'logged_in', 'per_page', 'category_instrumental', 'page', 'filterLabels2'])

In [56]:
d['meta_description']

21

In [ ]:
d['result_size']

In [ ]:
d['_embedded']

In [ ]:
d['_embedded'].keys()

In [126]:
d['_embedded']['estates']

[{'labelsReleased': [['balcony', 'garage', 'furnished'], []],
  'has_panorama': 0,
  'labels': ['Balkon', 'Garáž', 'Vybavený'],
  'is_auction': False,
  'labelsAll': [['personal',
    'balcony',
    'terrace',
    'cellar',
    'elevator',
    'parking_lots',
    'garage',
    'furnished'],
   ['small_shop',
    'theater',
    'vet',
    'candy_shop',
    'movies',
    'tavern',
    'playground',
    'sightseeing',
    'natural_attraction',
    'restaurant',
    'sports',
    'shop',
    'school',
    'drugstore',
    'metro',
    'medic',
    'kindergarten',
    'train',
    'post_office',
    'bus_public_transport',
    'tram',
    'atm']],
  'seo': {'category_main_cb': 1,
   'category_sub_cb': 8,
   'category_type_cb': 1,
   'locality': 'praha-zizkov-prokopova'},
  'exclusively_at_rk': 1,
  'category': 1,
  'has_floor_plan': 0,
  '_embedded': {'favourite': {'is_favourite': False,
    '_links': {'self': {'profile': '/favourite/doc',
      'href': '/cs/v2/favourite/1318314828',
      

### 1b. Create a function converting sreality json data into pandas dataframe

In [127]:
len(d['_embedded']['estates']) # number of estates

,labelsReleased,has_panorama,labels,is_auction,labelsAll,seo,exclusively_at_rk,category,has_floor_plan,_embedded,...,hash_id,attractive_offer,price,price_czk,_links,rus,name,region_tip,gps,has_matterport_url
0,"[[balcony, garage, furnished], []]",0,"[Balkon, Garáž, Vybavený]",False,"[[personal, balcony, terrace, cellar, elevator...","{'category_main_cb': 1, 'category_sub_cb': 8, ...",1,1,0,"{'favourite': {'is_favourite': False, '_links'...",...,1318314828,0,42769000,"{'value_raw': 42769000, 'unit': '', 'name': 'C...",{'dynamicDown': [{'href': 'https://d18-a.sdn.c...,False,Prodej bytu 4+kk 317 m²,3419754,"{'lat': 50.07261362854839, 'lon': 14.466933371...",False
1,"[[], []]",0,[],False,"[[new_building, personal, brick, elevator], [s...","{'category_main_cb': 1, 'category_sub_cb': 4, ...",0,1,1,"{'favourite': {'is_favourite': False, '_links'...",...,3481568076,0,36990000,"{'value_raw': 36990000, 'unit': '', 'name': 'C...",{'dynamicDown': [{'href': 'https://d18-a.sdn.c...,False,Prodej bytu 2+kk 130 m²,0,"{'lat': 50.06533562854839, 'lon': 14.459462371...",False
2,"[[not_furnished], []]",0,[Nevybavený],False,"[[new_building, personal, terrace, cellar, par...","{'category_main_cb': 1, 'category_sub_cb': 8, ...",1,1,1,"{'favourite': {'is_favourite': False, '_links'...",...,2106786380,0,33505000,"{'value_raw': 33505000, 'unit': '', 'name': 'C...",{'dynamicDown': [{'href': 'https://d18-a.sdn.c...,False,Prodej bytu 4+kk 379 m² (Mezonet),0,"{'lat': 50.12693562854839, 'lon': 14.390640371...",False
3,"[[], []]",0,[],False,"[[new_building, personal, terrace, brick, cell...","{'category_main_cb': 1, 'category_sub_cb': 4, ...",1,1,0,"{'favourite': {'is_favourite': False, '_links'...",...,1245102924,0,13300000,"{'value_raw': 13300000, 'unit': '', 'name': 'C...",{'dynamicDown': [{'href': 'https://d18-a.sdn.c...,False,Prodej bytu 2+kk 75 m²,0,"{'lat': 50.004075628548385, 'lon': 14.53868937...",False
4,"[[loggia], []]",0,[Lodžie],False,"[[new_building, personal, loggia, brick, cella...","{'category_main_cb': 1, 'category_sub_cb': 4, ...",0,1,1,"{'favourite': {'is_favourite': False, '_links'...",...,2240992076,0,12026000,"{'value_raw': 12026000, 'unit': '', 'name': 'C...",{'dynamicDown': [{'href': 'https://d18-a.sdn.c...,True,Prodej bytu 2+kk 67 m² (Jednopodlažní),0,"{'lat': 50.073510628548384, 'lon': 14.51542137...",False
5,"[[], []]",0,[],False,"[[personal, brick, cellar, elevator, partly_fu...","{'category_main_cb': 1, 'category_sub_cb': 5, ...",1,1,0,"{'favourite': {'is_favourite': False, '_links'...",...,3119788876,0,20781000,"{'value_raw': 20781000, 'unit': '', 'name': 'C...",{'dynamicDown': [{'href': 'https://d18-a.sdn.c...,False,Prodej bytu 2+1 130 m² (Jednopodlažní),0,"{'lat': 50.07916762854838, 'lon': 14.445585371...",False
6,"[[after_reconstruction, garage, not_furnished]...",0,"[Po rekonstrukci, Garáž, Nevybavený]",False,"[[personal, after_reconstruction, terrace, bri...","{'category_main_cb': 1, 'category_sub_cb': 9, ...",0,1,1,"{'favourite': {'is_favourite': False, '_links'...",...,1245131596,0,0,"{'value_raw': 0, 'unit': '', 'name': 'Celková ...",{'dynamicDown': [{'href': 'https://d18-a.sdn.c...,False,Prodej bytu 4+1 107 m²,0,"{'lat': 50.066052628548384, 'lon': 14.33694137...",False
7,"[[], []]",0,[],False,"[[personal, terrace, brick, cellar, elevator, ...","{'category_main_cb': 1, 'category_sub_cb': 4, ...",0,1,0,"{'favourite': {'is_favourite': False, '_links'...",...,1634022220,0,16759000,"{'value_raw': 16759000, 'unit': '', 'name': 'C...",{'dynamicDown': [{'href': 'https://d18-a.sdn.c...,False,Prodej bytu 2+kk 57 m²,0,"{'lat': 50.091805628548386, 'lon': 14.46582337...",False
8,"[[collective], [post_office]]",0,"[Družstevní, Pošta 5 min. pěšky]",False,"[[collective, brick, partly_furnished], [small...","{'category_main_cb': 1, 'category_sub_cb': 5, ...",0,1,1,"{'favourite': {'is_favourite': False, '_links'...",...,4202115916,0,12066000,"{'value_raw': 12066000, 'unit': '', 'name': 'C...",{'dynamicDown': [{'href': 'https://d18-a.sdn.

In [ ]:
pd.DataFrame(d['_embedded']['estates'])

In [59]:
def convert_sreality_data_to_df(sreality_data):
    data = sreality_data['_embedded']['estates']
    return pd.DataFrame(data)

raw = convert_sreality_data_to_df(d)

In [60]:
raw

,labelsReleased,has_panorama,labels,is_auction,labelsAll,seo,exclusively_at_rk,category,has_floor_plan,_embedded,...,hash_id,attractive_offer,price,price_czk,_links,rus,name,region_tip,gps,has_matterport_url
0,"[[balcony, garage, furnished], []]",0,"[Balkon, Garáž, Vybavený]",False,"[[personal, balcony, terrace, cellar, elevator...","{'category_main_cb': 1, 'category_sub_cb': 8, ...",1,1,0,"{'favourite': {'is_favourite': False, '_links'...",...,1318314828,0,42769000,"{'value_raw': 42769000, 'unit': '', 'name': 'C...",{'dynamicDown': [{'href': 'https://d18-a.sdn.c...,False,Prodej bytu 4+kk 317 m²,3419754,"{'lat': 50.07261362854839, 'lon': 14.466933371...",False
1,"[[], []]",0,[],False,"[[new_building, personal, brick, elevator], [s...","{'category_main_cb': 1, 'category_sub_cb': 4, ...",0,1,1,"{'favourite': {'is_favourite': False, '_links'...",...,3481568076,0,36990000,"{'value_raw': 36990000, 'unit': '', 'name': 'C...",{'dynamicDown': [{'href': 'https://d18-a.sdn.c...,False,Prodej bytu 2+kk 130 m²,0,"{'lat': 50.06533562854839, 'lon': 14.459462371...",False
2,"[[not_furnished], []]",0,[Nevybavený],False,"[[new_building, personal, terrace, cellar, par...","{'category_main_cb': 1, 'category_sub_cb': 8, ...",1,1,1,"{'favourite': {'is_favourite': False, '_links'...",...,2106786380,0,33505000,"{'value_raw': 33505000, 'unit': '', 'name': 'C...",{'dynamicDown': [{'href': 'https://d18-a.sdn.c...,False,Prodej bytu 4+kk 379 m² (Mezonet),0,"{'lat': 50.12693562854839, 'lon': 14.390640371...",False
3,"[[], []]",0,[],False,"[[new_building, personal, terrace, brick, cell...","{'category_main_cb': 1, 'category_sub_cb': 4, ...",1,1,0,"{'favourite': {'is_favourite': False, '_links'...",...,1245102924,0,13300000,"{'value_raw': 13300000, 'unit': '', 'name': 'C...",{'dynamicDown': [{'href': 'https://d18-a.sdn.c...,False,Prodej bytu 2+kk 75 m²,0,"{'lat': 50.004075628548385, 'lon': 14.53868937...",False
4,"[[loggia], []]",0,[Lodžie],False,"[[new_building, personal, loggia, brick, cella...","{'category_main_cb': 1, 'category_sub_cb': 4, ...",0,1,1,"{'favourite': {'is_favourite': False, '_links'...",...,2240992076,0,12026000,"{'value_raw': 12026000, 'unit': '', 'name': 'C...",{'dynamicDown': [{'href': 'https://d18-a.sdn.c...,True,Prodej bytu 2+kk 67 m² (Jednopodlažní),0,"{'lat': 50.073510628548384, 'lon': 14.51542137...",False
5,"[[], []]",0,[],False,"[[personal, brick, cellar, elevator, partly_fu...","{'category_main_cb': 1, 'category_sub_cb': 5, ...",1,1,0,"{'favourite': {'is_favourite': False, '_links'...",...,3119788876,0,20781000,"{'value_raw': 20781000, 'unit': '', 'name': 'C...",{'dynamicDown': [{'href': 'https://d18-a.sdn.c...,False,Prodej bytu 2+1 130 m² (Jednopodlažní),0,"{'lat': 50.07916762854838, 'lon': 14.445585371...",False
6,"[[after_reconstruction, garage, not_furnished]...",0,"[Po rekonstrukci, Garáž, Nevybavený]",False,"[[personal, after_reconstruction, terrace, bri...","{'category_main_cb': 1, 'category_sub_cb': 9, ...",0,1,1,"{'favourite': {'is_favourite': False, '_links'...",...,1245131596,0,0,"{'value_raw': 0, 'unit': '', 'name': 'Celková ...",{'dynamicDown': [{'href': 'https://d18-a.sdn.c...,False,Prodej bytu 4+1 107 m²,0,"{'lat': 50.066052628548384, 'lon': 14.33694137...",False
7,"[[], []]",0,[],False,"[[personal, terrace, brick, cellar, elevator, ...","{'category_main_cb': 1, 'category_sub_cb': 4, ...",0,1,0,"{'favourite': {'is_favourite': False, '_links'...",...,1634022220,0,16759000,"{'value_raw': 16759000, 'unit': '', 'name': 'C...",{'dynamicDown': [{'href': 'https://d18-a.sdn.c...,False,Prodej bytu 2+kk 57 m²,0,"{'lat': 50.091805628548386, 'lon': 14.46582337...",False
8,"[[collective], [post_office]]",0,"[Družstevní, Pošta 5 min. pěšky]",False,"[[collective, brick, partly_furnished], [small...","{'category_main_cb': 1, 'category_sub_cb': 5, ...",0,1,1,"{'favourite': {'is_favourite': False, '_links'...",...,4202115916,0,12066000,"{'value_raw': 12066000, 'unit': '', 'name': 'C...",{'dynamicDown': [{'href': 'https://d18-a.sdn.

### 1c. link function `1b` into function `1a`

In [61]:
df = convert_sreality_data_to_df(request_sreality(0, 'flat', 'sell', 10))
df.head()

,labelsReleased,has_panorama,labels,is_auction,labelsAll,seo,exclusively_at_rk,category,has_floor_plan,_embedded,...,hash_id,attractive_offer,price,price_czk,_links,rus,name,region_tip,gps,has_matterport_url
0,"[[parking_lots, furnished], []]",0,"[Parkování, Vybavený]",False,"[[personal, balcony, brick, cellar, elevator, ...","{'category_main_cb': 1, 'category_sub_cb': 3, ...",0,1,0,"{'favourite': {'is_favourite': False, '_links'...",...,722592588,0,9787000,"{'value_raw': 9787000, 'unit': '', 'name': 'Ce...",{'dynamicDown': [{'href': 'https://d18-a.sdn.c...,False,Prodej bytu 1+1 60 m²,3418491,"{'lat': 50.12930762854839, 'lon': 14.588600371...",False
1,"[[new_building, parking_lots], [shop]]",0,"[Novostavba, Parkování, Obchod 6 min. pěšky]",False,"[[new_building, personal, balcony, cellar, ele...","{'category_main_cb': 1, 'category_sub_cb': 6, ...",1,1,1,"{'favourite': {'is_favourite': False, '_links'...",...,4070581580,0,16366000,"{'value_raw': 16366000, 'unit': '', 'name': 'C...",{'dynamicDown': [{'href': 'https://d18-a.sdn.c...,False,Prodej bytu 3+kk 103 m²,0,"{'lat': 50.14428862854839, 'lon': 14.532566371...",False
2,"[[new_building, terrace, furnished], [metro]]",0,"[Novostavba, Terasa, Vybavený, Metro 5 min. pě...",False,"[[new_building, personal, balcony, terrace, br...","{'category_main_cb': 1, 'category_sub_cb': 6, ...",1,1,1,"{'favourite': {'is_favourite': False, '_links'...",...,687641420,0,42863000,"{'value_raw': 42863000, 'unit': '', 'name': 'C...",{'dynamicDown': [{'href': 'https://d18-a.sdn.c...,False,Prodej bytu 3+kk 160 m² (Mezonet),0,"{'lat': 50.06318362854839, 'lon': 14.448837371...",False
3,"[[in_construction, terrace], []]",0,"[Ve výstavbě, Terasa]",False,"[[in_construction, personal, terrace], [candy_...","{'category_main_cb': 1, 'category_sub_cb': 4, ...",1,1,1,"{'favourite': {'is_favourite': False, '_links'...",...,3349996364,0,19360000,"{'value_raw': 19360000, 'unit': '', 'name': 'C...",{'dynamicDown': [{'href': 'https://d18-a.sdn.c...,False,Prodej bytu 2+kk 97 m² (Mezonet),0,"{'lat': 50.024453628548386, 'lon': 14.39995737...",False
4,"[[after_reconstruction, partly_furnished], []]",0,"[Po rekonstrukci, Částečně vybavený]",False,"[[personal, after_reconstruction, balcony, bri...","{'category_main_cb': 1, 'category_sub_cb': 9, ...",0,1,1,"{'favourite': {'is_favourite': False, '_links'...",...,887763788,0,38699000,"{'value_raw': 38699000, 'unit': '', 'name': 'C...",{'dynamicDown': [{'href': 'https://d18-a.sdn.c...,False,Prodej bytu 4+1 189 m²,0,"{'lat': 50.05999162854839, 'lon': 14.421697371...",False


In [62]:
df.shape

(21, 27)

### 1c. Combining multiple requests into single df

* Function should parametrize:
    * `start_page` and `end_page`
    * request parameters
* construct a list of individual request dfs
* then feed it into `pd.concat` function

In [ ]:
raw.shape

In [63]:
def request_multiple_sreality(start_page, end_page, category_main_str, category_type_str, locality_region_id=10):
    visit_pages = range(start_page, end_page)
    list_of_dfs = []
    for page in visit_pages:
        df = convert_sreality_data_to_df(request_sreality(page, category_main_str, category_type_str, locality_region_id))
        list_of_dfs.append(df)
    return pd.concat(list_of_dfs)

In [64]:
df = request_multiple_sreality(1, 5, 'flat', 'sell', locality_region_id=10)

In [66]:
df.shape

(84, 27)

In [69]:
df.head()

,labelsReleased,has_panorama,labels,is_auction,labelsAll,seo,exclusively_at_rk,category,has_floor_plan,_embedded,...,hash_id,attractive_offer,price,price_czk,_links,rus,name,region_tip,gps,has_matterport_url
0,"[[parking_lots, furnished], []]",0,"[Parkování, Vybavený]",False,"[[personal, balcony, brick, cellar, elevator, ...","{'category_main_cb': 1, 'category_sub_cb': 3, ...",0,1,0,"{'favourite': {'is_favourite': False, '_links'...",...,722592588,0,9787000,"{'value_raw': 9787000, 'unit': '', 'name': 'Ce...",{'dynamicDown': [{'href': 'https://d18-a.sdn.c...,False,Prodej bytu 1+1 60 m²,3418491,"{'lat': 50.12930762854839, 'lon': 14.588600371...",False
1,"[[new_building, parking_lots], [shop]]",0,"[Novostavba, Parkování, Obchod 6 min. pěšky]",False,"[[new_building, personal, balcony, cellar, ele...","{'category_main_cb': 1, 'category_sub_cb': 6, ...",1,1,1,"{'favourite': {'is_favourite': False, '_links'...",...,4070581580,0,16366000,"{'value_raw': 16366000, 'unit': '', 'name': 'C...",{'dynamicDown': [{'href': 'https://d18-a.sdn.c...,False,Prodej bytu 3+kk 103 m²,0,"{'lat': 50.14428862854839, 'lon': 14.532566371...",False
2,"[[new_building, terrace, furnished], [metro]]",0,"[Novostavba, Terasa, Vybavený, Metro 5 min. pě...",False,"[[new_building, personal, balcony, terrace, br...","{'category_main_cb': 1, 'category_sub_cb': 6, ...",1,1,1,"{'favourite': {'is_favourite': False, '_links'...",...,687641420,0,42863000,"{'value_raw': 42863000, 'unit': '', 'name': 'C...",{'dynamicDown': [{'href': 'https://d18-a.sdn.c...,False,Prodej bytu 3+kk 160 m² (Mezonet),0,"{'lat': 50.06318362854839, 'lon': 14.448837371...",False
3,"[[in_construction, terrace], []]",0,"[Ve výstavbě, Terasa]",False,"[[in_construction, personal, terrace], [candy_...","{'category_main_cb': 1, 'category_sub_cb': 4, ...",1,1,1,"{'favourite': {'is_favourite': False, '_links'...",...,3349996364,0,19360000,"{'value_raw': 19360000, 'unit': '', 'name': 'C...",{'dynamicDown': [{'href': 'https://d18-a.sdn.c...,False,Prodej bytu 2+kk 97 m² (Mezonet),0,"{'lat': 50.024453628548386, 'lon': 14.39995737...",False
4,"[[after_reconstruction, partly_furnished], []]",0,"[Po rekonstrukci, Částečně vybavený]",False,"[[personal, after_reconstruction, balcony, bri...","{'category_main_cb': 1, 'category_sub_cb': 9, ...",0,1,1,"{'favourite': {'is_favourite': False, '_links'...",...,887763788,0,38699000,"{'value_raw': 38699000, 'unit': '', 'name': 'C...",{'dynamicDown': [{'href': 'https://d18-a.sdn.c...,False,Prodej bytu 4+1 189 m²,0,"{'lat': 50.05999162854839, 'lon': 14.421697371...",False


In [68]:
# reset the index
df.reset_index(drop=True, inplace=True)

In [ ]:
df.index

#### Task 2: Cleaning data

__2a. Filter columns__
* filter only columns: `['locality', 'price', 'name', 'gps','hash_id','exclusively_at_rk']`
* use `.copy()` to avoid `SettingWithCopyWarning` later


In [70]:
# display the columns
df.columns

Index(['labelsReleased', 'has_panorama', 'labels', 'is_auction', 'labelsAll',
       'seo', 'exclusively_at_rk', 'category', 'has_floor_plan', '_embedded',
       'paid_logo', 'locality', 'has_video', 'advert_images_count', 'new',
       'auctionPrice', 'type', 'hash_id', 'attractive_offer', 'price',
       'price_czk', '_links', 'rus', 'name', 'region_tip', 'gps',
       'has_matterport_url'],
      dtype='object')

Copy selection into new one

In [71]:
columns_to_keep = ['locality', 'price', 'name', 'gps','hash_id','exclusively_at_rk']
df_clean = df[columns_to_keep].copy()
df_clean.head()

,locality,price,name,gps,hash_id,exclusively_at_rk
0,Praha 9 - Vinoř,9787000,Prodej bytu 1+1 60 m²,"{'lat': 50.12930762854839, 'lon': 14.588600371...",722592588,0
1,Praha 9 - Čakovice,16366000,Prodej bytu 3+kk 103 m²,"{'lat': 50.14428862854839, 'lon': 14.532566371...",4070581580,1
2,Praha 2 - Vinohrady,42863000,Prodej bytu 3+kk 160 m² (Mezonet),"{'lat': 50.06318362854839, 'lon': 14.448837371...",687641420,1
3,Praha 5 - Hlubočepy,19360000,Prodej bytu 2+kk 97 m² (Mezonet),"{'lat': 50.024453628548386, 'lon': 14.39995737...",3349996364,1
4,Praha 5 - Smíchov,38699000,Prodej bytu 4+1 189 m²,"{'lat': 50.05999162854839, 'lon': 14.421697371...",887763788,0


### 2b: GPS
* Convert dictionary in `gps` column into two columns - `lat` and `lon`
* use apply function on gps column
* Note apply can return multiple columns

In [79]:
df_clean.loc[1:2, 'gps']

In [83]:
df_clean[['lat', 'lon']] = df_clean.gps.apply(lambda x: pd.Series({'lat': x['lat'], 'lon': x['lon']}))

,locality,price,name,gps,hash_id,exclusively_at_rk,lon,lat
0,Praha 9 - Vinoř,9787000,Prodej bytu 1+1 60 m²,"{'lat': 50.12930762854839, 'lon': 14.588600371...",722592588,0,14.588600,50.129308
1,Praha 9 - Čakovice,16366000,Prodej bytu 3+kk 103 m²,"{'lat': 50.14428862854839, 'lon': 14.532566371...",4070581580,1,14.532566,50.144289
2,Praha 2 - Vinohrady,42863000,Prodej bytu 3+kk 160 m² (Mezonet),"{'lat': 50.06318362854839, 'lon': 14.448837371...",687641420,1,14.448837,50.063184
3,Praha 5 - Hlubočepy,19360000,Prodej bytu 2+kk 97 m² (Mezonet),"{'lat': 50.024453628548386, 'lon': 14.39995737...",3349996364,1,14.399957,50.024454
4,Praha 5 - Smíchov,38699000,Prodej bytu 4+1 189 m²,"{'lat': 50.05999162854839, 'lon': 14.421697371...",887763788,0,14.421697,50.059992


In [ ]:
df_clean.head()

### 2c. Get flat type from name

* Name is always represented by string `Prodej bytu [type of flat] [Area] m^2`
* Try picking third word in string
* Check meaningfulness using `.value_counts()`

In [90]:
df_clean.name[0]

'Prodej bytu 1+1\xa060\xa0m²'

In [94]:
df_clean.name[0].split(' ')

['Prodej', 'bytu', '1+1\xa060\xa0m²']

In [103]:
# type of flat (' ')
df_clean['type_of_flat'] = df_clean.name.apply(lambda x: x.split(' ')[2])

In [96]:
# flat_type ()
df_clean['flat_type'] = df_clean.name.apply(lambda x: x.split()[2])


In [101]:
df_clean['flat_type'].value_counts()

flat_type
2+kk        27
3+kk        16
4+kk        10
2+1          9
4+1          7
3+1          5
1+1          4
1+kk         4
atypické     1
5+1          1
Name: count, dtype: int64

In [102]:
df_clean['type_of_flat'].value_counts()

type_of_flat
2+kk          27
3+kk          16
4+kk          10
1+kk           4
1+1 60 m²      2
2+1 111 m²     2
4+1 107 m²     2
2+1 80 m²      2
3+1 103 m²     1
atypické       1
4+1 189 m²     1
2+1 95 m²      1
3+1 117 m²     1
3+1 96 m²      1
3+1 130 m²     1
2+1 96 m²      1
4+1 290 m²     1
1+1 73 m²      1
4+1 160 m²     1
1+1 50 m²      1
4+1 117 m²     1
4+1 130 m²     1
2+1 130 m²     1
2+1 85 m²      1
3+1 107 m²     1
5+1 214 m²     1
2+1 72 m²      1
Name: count, dtype: int64

### 2d. Get the area of a flat from name

* Naive: select the word before last word - save as `area1`
* Then try navigating using the index of `'m²'` in the splitted string - save as `area2`
* If this also fail, then you will need to use regex - `import re` - save as `area3`

In [118]:
df_clean['area1'] = df_clean.name.apply(lambda x: int(x.split()[3]))
df_clean.head()

,locality,price,name,gps,hash_id,exclusively_at_rk,lon,lat,type_of_flat,flat_type,area1,area2,area3
0,Praha 9 - Vinoř,9787000,Prodej bytu 1+1 60 m²,"{'lat': 50.12930762854839, 'lon': 14.588600371...",722592588,0,14.588600,50.129308,1+1 60 m²,1+1,60,60,60
1,Praha 9 - Čakovice,16366000,Prodej bytu 3+kk 103 m²,"{'lat': 50.14428862854839, 'lon': 14.532566371...",4070581580,1,14.532566,50.144289,3+kk,3+kk,103,103,103
2,Praha 2 - Vinohrady,42863000,Prodej bytu 3+kk 160 m² (Mezonet),"{'lat': 50.06318362854839, 'lon': 14.448837371...",687641420,1,14.448837,50.063184,3+kk,3+kk,160,160,160
3,Praha 5 - Hlubočepy,19360000,Prodej bytu 2+kk 97 m² (Mezonet),"{'lat': 50.024453628548386, 'lon': 14.39995737...",3349996364,1,14.399957,50.024454,2+kk,2+kk,97,97,97
4,Praha 5 - Smíchov,38699000,Prodej bytu 4+1 189 m²,"{'lat': 50.05999162854839, 'lon': 14.421697371...",887763788,0,14.421697,50.059992,4+1 189 m²,4+1,189,189,189


In [119]:
def name_to_area(nm):
    splitted_str = nm.split()
    m2_idx = splitted_str.index('m²')
    return int(splitted_str[m2_idx - 1])

df_clean['area2'] = df_clean.name.apply(name_to_area)
df_clean.head()

,locality,price,name,gps,hash_id,exclusively_at_rk,lon,lat,type_of_flat,flat_type,area1,area2,area3
0,Praha 9 - Vinoř,9787000,Prodej bytu 1+1 60 m²,"{'lat': 50.12930762854839, 'lon': 14.588600371...",722592588,0,14.588600,50.129308,1+1 60 m²,1+1,60,60,60
1,Praha 9 - Čakovice,16366000,Prodej bytu 3+kk 103 m²,"{'lat': 50.14428862854839, 'lon': 14.532566371...",4070581580,1,14.532566,50.144289,3+kk,3+kk,103,103,103
2,Praha 2 - Vinohrady,42863000,Prodej bytu 3+kk 160 m² (Mezonet),"{'lat': 50.06318362854839, 'lon': 14.448837371...",687641420,1,14.448837,50.063184,3+kk,3+kk,160,160,160
3,Praha 5 - Hlubočepy,19360000,Prodej bytu 2+kk 97 m² (Mezonet),"{'lat': 50.024453628548386, 'lon': 14.39995737...",3349996364,1,14.399957,50.024454,2+kk,2+kk,97,97,97
4,Praha 5 - Smíchov,38699000,Prodej bytu 4+1 189 m²,"{'lat': 50.05999162854839, 'lon': 14.421697371...",887763788,0,14.421697,50.059992,4+1 189 m²,4+1,189,189,189


In [120]:
# regular expression to find characters before '+' and behind '+'
def flat_type(x:str):
    # \d+ represents one or more digit(s)
    # \s is a space character
    # \w+ is one or more word character(s)
    # * represents 0 or more
    ft = re.findall(r'(\d+)\s*\+\s*(\w+)', x)
    return ft

In [121]:
sample_text = 'Prodej bytu 3+kk 71 m²'
print(f"Initial text: {sample_text}")
print(f"Processed text: {flat_type(sample_text)}")


Initial text: Prodej bytu 3+kk 71 m²
Processed text: [('3', 'kk')]


In [122]:
df_clean['area3'] = df_clean.name.apply(lambda x: int(re.findall(r'(\d+)\s*m²', x)[0]))
df_clean.head()

,locality,price,name,gps,hash_id,exclusively_at_rk,lon,lat,type_of_flat,flat_type,area1,area2,area3
0,Praha 9 - Vinoř,9787000,Prodej bytu 1+1 60 m²,"{'lat': 50.12930762854839, 'lon': 14.588600371...",722592588,0,14.588600,50.129308,1+1 60 m²,1+1,60,60,60
1,Praha 9 - Čakovice,16366000,Prodej bytu 3+kk 103 m²,"{'lat': 50.14428862854839, 'lon': 14.532566371...",4070581580,1,14.532566,50.144289,3+kk,3+kk,103,103,103
2,Praha 2 - Vinohrady,42863000,Prodej bytu 3+kk 160 m² (Mezonet),"{'lat': 50.06318362854839, 'lon': 14.448837371...",687641420,1,14.448837,50.063184,3+kk,3+kk,160,160,160
3,Praha 5 - Hlubočepy,19360000,Prodej bytu 2+kk 97 m² (Mezonet),"{'lat': 50.024453628548386, 'lon': 14.39995737...",3349996364,1,14.399957,50.024454,2+kk,2+kk,97,97,97
4,Praha 5 - Smíchov,38699000,Prodej bytu 4+1 189 m²,"{'lat': 50.05999162854839, 'lon': 14.421697371...",887763788,0,14.421697,50.059992,4+1 189 m²,4+1,189,189,189


__Check if the three areas are equal__

In [128]:
(df_clean['area1'] == df_clean['area2']).all()

np.True_